In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    li

In [2]:
!conda install -c conda-forge BeautifulSoup4 --yes

!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.4            |           py36_0          58 KB  conda-forge
    beautifulsoup4-4.8.1       |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         207 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.4-py36_0 conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.6.3-py37_0             --> 4.8.1-py36_0 conda-forge


soupsieve-1.9.4     

In [125]:
from bs4 import BeautifulSoup # import the BeautifulSoup library so we can parse HTML and XML documents
import urllib.request # import the library we use to open URLs
import xml.etree.ElementTree as ET
import requests

# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# parse the HTML from our URL into the BeautifulSoup parse tree format
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')

#processing the cells that have an assigned borough and Ignoring cells with a borough that is Not assigned.

lst_table = []
for row in table.find_all('tr'):
    if (row.contents[3].text != 'Not assigned'):
        lst_table.append([row.contents[1].text, row.contents[3].text, row.contents[5].text[:-1]])

lst_table

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M9A', "Queen's Park", "Queen's Park"],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 'Etobicoke', 'Martin Grove'],
 ['M9B', 'Etobicoke', 'Princess Gardens'],
 ['M9B', 'Etobicoke', 'West Deane Park'],
 ['M1C', 'Scarborough', 'Highland Creek'],
 ['M1C', 'Scarborough', 'Rouge Hill'],
 ['M1C', 'Scarborough', 'Port Union'],
 ['M3C', 'North Yor

In [123]:
       
df_postal = pd.DataFrame(lst_table[1:], columns=lst_table[0])
df_code = df_postal['Postcode'].unique()
df_postal.set_index('Postcode', drop=False, inplace=True)

separator=', '
for postcode in df_code:
    df_tmp = df_postal.loc[postcode]
    if(df_tmp.size > 3):
        neighbourhood = separator.join(df_tmp['Neighbourhood'])
        df_postal.loc[postcode, 'Neighbourhood'] = neighbourhood

df_postal.drop(df_postal[df_postal.Neighbourhood == 'Not assigned'].index, inplace=True)
df_postal.drop_duplicates('Postcode', inplace=True)
df_postal.reset_index(drop = True, inplace=True)
df_postal.head(10)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M9A,Queen's Park,Queen's Park
5,M1B,Scarborough,"Rouge, Malvern"
6,M3B,North York,Don Mills North
7,M4B,East York,"Woodbine Gardens, Parkview Hill"
8,M5B,Downtown Toronto,"Ryerson, Garden District"
9,M6B,North York,Glencairn


In [117]:
print('Number of records in the data frame is: ', df_postal.shape)

Number of records in the data frame is:  (102, 3)


In [147]:
#Use geopy library to get the latitude and longitude values of New York City.
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="cot_explorer")
location = geolocator.geocode(address)
print(location)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

Toronto, Golden Horseshoe, Ontario, M6K 1X9, Canada
The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [134]:
D = []
E = []

for index, row in df_postal.iterrows():
    address = row['Borough']
    geolocator = Nominatim(user_agent="cot_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    D.append(format(latitude))
    E.append(format(longitude))
    
df_postal["Latitude"] = D
df_postal["Longtitude"] = E
 

In [142]:
df_postal.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longtitude
0,M3A,North York,Parkwoods,43.7708175,-79.4132998
1,M4A,North York,Victoria Village,43.7708175,-79.4132998
2,M5A,Downtown Toronto,Harbourfront,43.654027,-79.3802003
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7708175,-79.4132998
4,M9A,Queen's Park,Queen's Park,43.6599803,-79.3903686
5,M1B,Scarborough,"Rouge, Malvern",54.2820009,-0.4011868
6,M3B,North York,Don Mills North,43.7708175,-79.4132998
7,M4B,East York,"Woodbine Gardens, Parkview Hill",43.6913391,-79.3278212
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.654027,-79.3802003
9,M6B,North York,Glencairn,43.7708175,-79.4132998


In [145]:
toronto_data = df_postal[df_postal['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longtitude
0,M5A,Downtown Toronto,Harbourfront,43.654027,-79.3802003
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.654027,-79.3802003
2,M5C,Downtown Toronto,St. James Town,43.654027,-79.3802003
3,M4E,East Toronto,The Beaches,43.653963,-79.387207
4,M5E,Downtown Toronto,Berczy Park,43.654027,-79.3802003
5,M5G,Downtown Toronto,Central Bay Street,43.654027,-79.3802003
6,M6G,Downtown Toronto,Christie,43.654027,-79.3802003
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.654027,-79.3802003
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.653963,-79.387207
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.654027,-79.3802003


In [156]:
#Use geopy library to get the latitude and longitude values of New York City.
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="cot_explorer")
location = geolocator.geocode(address)
print(location)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))
print(type(latitude))

Toronto, Golden Horseshoe, Ontario, M6K 1X9, Canada
The geograpical coordinate of Toronto City are 43.653963, -79.387207.
<class 'float'>


In [158]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longtitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [159]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'RRFTODUHGCMU4R12HWRMSLGTIT4CIQAEFMCLCOY345S35LPM' # your Foursquare ID
CLIENT_SECRET = 'XVG2KQVYBWXWLBYBRSYK5L3HZIFSFD3SQXWQZ1TEE10RT3HC' # your Foursquare Secret
VERSION = '20191017' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RRFTODUHGCMU4R12HWRMSLGTIT4CIQAEFMCLCOY345S35LPM
CLIENT_SECRET:XVG2KQVYBWXWLBYBRSYK5L3HZIFSFD3SQXWQZ1TEE10RT3HC


In [161]:
toronto_data.loc[0, 'Neighbourhood']

'Harbourfront'

In [169]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 500
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longtitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longtitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [170]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longtitude']
                                  )


Harbourfront
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian P

In [171]:
print(toronto_venues.shape)
toronto_venues.head()

(3360, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longtitude,Venue,Venue Latitude,Venue Longtitude,Venue Category
0,Harbourfront,43.654027,-79.3802003,Elgin And Winter Garden Theatres,43.653394,-79.378507,Theater
1,Harbourfront,43.654027,-79.3802003,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store
2,Harbourfront,43.654027,-79.3802003,Indigo,43.653515,-79.380696,Bookstore
3,Harbourfront,43.654027,-79.3802003,Ed Mirvish Theatre,43.655102,-79.379768,Theater
4,Harbourfront,43.654027,-79.3802003,LUSH,43.653557,-79.380400,Cosmetics Shop


In [172]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longtitude,Venue,Venue Latitude,Venue Longtitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",78,78,78,78,78,78
Business Reply Mail Processing Centre 969 Eastern,78,78,78,78,78,78
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",100,100,100,100,100,100
"Cabbagetown, St. James Town",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


In [173]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 81 uniques categories.


In [174]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Furniture / Home Store,Gastropub,Gift Shop,Greek Restaurant,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Neighborhood,New American Restaurant,Office,Opera House,Park,Pizza Place,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [175]:
toronto_onehot.shape

(3360, 82)

In [176]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Furniture / Home Store,Gastropub,Gift Shop,Greek Restaurant,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Neighborhood,New American Restaurant,Office,Opera House,Park,Pizza Place,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,University,Vegetarian / Vegan Restaurant
0,"Adelaide, King, Richmond",0.030000,0.000000,0.000000,0.020000,0.040000,0.01,0.010000,0.02,0.02,0.020000,0.020000,0.02,0.01,0.010000,0.020000,0.050000,0.050000,0.01,0.000000,0.030000,0.000000,0.01,0.010000,0.01,0.000000,0.01,0.03,0.01,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.020000,0.000000,0.030000,0.010000,0.000000,0.03,0.010000,0.000000,0.000000,0.000000,0.01,0.01,0.01,0.02,0.010000,0.010000,0.010000,0.010000,0.010000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.000000,0.010000,0.000000,0.040000,0.000000,0.000000,0.020000,0.01,0.000000,0.01,0.01,0.02,0.010000,0.030000,0.000000,0.020000,0.010000,0.02,0.010000,0.000000,0.000000
1,Berczy Park,0.030000,0.000000,0.000000,0.020000,0.040000,0.01,0.010000,0.02,0.02,0.020000,0.020000,0.02,0.01,0.010000,0.020000,0.050000,0.050000,0.01,0.000000,0.030000,0.000000,0.01,0.010000,0.01,0.000000,0.01,0.03,0.01,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.020000,0.000000,0.030000,0.010000,0.000000,0.03,0.010000,0.000000,0.000000,0.000000,0.01,0.01,0.01,0.02,0.010000,0.010000,0.010000,0.010000,0.010000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.000000,0.010000,0.000000,0.040000,0.000000,0.000000,0.020000,0.01,0.000000,0.01,0.01,0.02,0.010000,0.030000,0.000000,0.020000,0.010000,0.02,0.010000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.012821,0.038462,0.012821,0.012821,0.012821,0.00,0.025641,0.00,0.00,0.038462,0.025641,0.00,0.00,0.051282,0.025641,0.012821,0.051282,0.00,0.012821,0.012821,0.012821,0.00,0.012821,0.00,0.012821,0.00,0.00,0.00,0.012821,0.012821,0.00,0.012821,0.012821,0.012821,0.012821,0.012821,0.025641,0.025641,0.012821,0.00,0.038462,0.012821,0.012821,0.012821,0.00,0.00,0.00,0.00,0.012821,0.012821,0.012821,0.012821,0.012821,0.00,0.012821,0.012821,0.012821,0.00,0.012821,0.012821,0.012821,0.025641,0.012821,0.012821,0.012821,0.012821,0.012821,0.00,0.012821,0.00,0.00,0.00,0.012821,0.038462,0.012821,0.012821,0.012821,0.00,0.012821,0.012821,0.025641
3,Business Reply Mail Processing Centre 969 Eastern,0.012821,0.038462,0.012821,0.012821,0.012821,0.00,0.025641,0.00,0.00,0.038462,0.025641,0.00,0.00,0.051282,0.025641,0.012821,0.051282,0.00,0.012821,0.012821,0.012821,0.00,0.012821,0.00,0.012821,0.00,0.00,0.00,0.012821,0.012821,0.00,0.012821,0.012821,0.012821,0.012821,0.012821,0.025641,0.025641,0.012821,0.00,0.038462,0.012821,0.012821,0.012821,0.00,0.00,0.00,0.00,0.012821,0.012821,0.012821,0.012821,0.012821,0.00,0.012821,0.012821,0.012821,0.00,0.012821,0.012821,0.012821,0.025641,0.012821,0.012821,0.012821,0.012821,0.012821,0.00,0.012821,0.00,0.00,0.00,0.012821,0.038462,0.012821,0.012821,0.012821,0.00,0.012821,0.012821,0.025641
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.030000,0.000000,0.000000,0.020000,0.040000,0.01,0.010000,0.02,0.02,0.020000,0.020000,0.02,0.01,0.01000

In [177]:
toronto_grouped.shape

(38, 82)

In [182]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    #print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [180]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [184]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Clothing Store,Restaurant,Bakery,American Restaurant,Hotel,Sushi Restaurant,Fast Food Restaurant,Cosmetics Shop,Italian Restaurant
1,Berczy Park,Coffee Shop,Clothing Store,Restaurant,Bakery,American Restaurant,Hotel,Sushi Restaurant,Fast Food Restaurant,Cosmetics Shop,Italian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Japanese Restaurant,Art Gallery,Breakfast Spot,Sushi Restaurant,Ramen Restaurant,Bar,Bubble Tea Shop,Chinese Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Café,Coffee Shop,Japanese Restaurant,Art Gallery,Breakfast Spot,Sushi Restaurant,Ramen Restaurant,Bar,Bubble Tea Shop,Chinese Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Clothing Store,Restaurant,Bakery,American Restaurant,Hotel,Sushi Restaurant,Fast Food Restaurant,Cosmetics Shop,Italian Restaurant


In [193]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return_n_iter=True)


array([3, 3, 0, 0, 3, 3, 3, 3, 3, 3], dtype=int32)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

In [199]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longtitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters